In [19]:
import sys
sys.path.append('../scripts/lib')

In [21]:
import csv
from PastecLib import PastecConnection
from os.path import isfile
from SPARQLWrapper import SPARQLWrapper, JSON

In [12]:
sparqlEndpoint = "http://localhost:8888/sparql"
pastecEndpoint = "http://localhost:4212"
datafile = "../data/imageData.csv"

In [3]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in results["head"]["vars"]:
            if key in result:
                row[key] = result[key]["value"]
            else:
                row[key] = None
        rows.append(row)
    return rows

## Retrieve data

In [4]:
data = []
if isfile(datafile):
    with open(datafile, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)

In [5]:
imagesQuery = """
PREFIX la: <https://linked.art/ns/terms/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX search: <https://platform.swissartresearch.net/search/>
SELECT ?subject ?image WHERE {
 ?subject a search:Object ;
     crm:P128_carries/la:digitally_shown_by/la:digitally_available_via/la:access_point ?image 
} 
ORDER BY ?subject 
LIMIT 15
"""

In [6]:
sparql = SPARQLWrapper(sparqlEndpoint, returnFormat=JSON)

In [7]:
sparql.setQuery(imagesQuery)
try:
    ret = sparql.query().convert()
except:
    raise Exception("Could not execute query against endpoint", endpoint)
imageDataFromStore = sparqlResultToDict(ret)

In [8]:
irisInData = [d['subject'] for d in data]

In [9]:
for row in imageDataFromStore:
    if row['subject'] not in irisInData:
        data.append({
            "index": len(data),
            "subject": row['subject'],
            "image": row['image']
        })

In [10]:
with open(datafile, 'w') as f:
    headers = ['index', 'subject', 'image']
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for row in data:
        writer.writerow(row)

## Submit to index